In [3]:
import pandas as pd
#from psaw import PushshiftAPI    #library Pushshift
import datetime as dt            #library for date management
#import p                         #library for data manipulation
import matplotlib.pyplot as plt  #library for plotting

In [2]:
api = PushshiftAPI()              #Object of the API

In [3]:
"""FOR POSTS"""
def data_prep_posts(subreddit, start_time, end_time, filters, limit):
    if(len(filters) == 0):
        filters = ['id', 'author', 'created_utc',
                   'domain', 'url',
                   'title', 'num_comments', 'thumbnail']                 
                   #We set by default some useful columns

    posts = list(api.search_submissions(
        subreddit=subreddit,   #Subreddit we want to audit
        after=start_time,      #Start date
        before=end_time,       #End date
        filter=filters,        #Column names we want to retrieve
        limit=limit))          ##Max number of posts

    return pd.DataFrame(posts) #Return dataframe for analysis

In [4]:
def get_subreddit_data_and_save_to_csv(subreddit, day_start, month_start, year_start, day_end, month_end, year_end, file_name):
    #subreddit = "TodayIAte"                           #Name of the subreddit we are auditing
    start_time = int(dt.datetime(year_start, month_start, day_start).timestamp())  #We define the starting date for our search
    end_time = int(dt.datetime(year_end, month_end, day_end).timestamp())   #We define the ending date for our search
    filters = []                                           #We don´t want specific filters
    limit = None   #Number of elelemts we want to recieve

    """Here we are going to get subreddits for a brief analysis"""
    df_p = data_prep_posts(subreddit,start_time,
                             end_time,filters,limit)           #Call function for dataframe creation of comments
    df_p['datetime'] = df_p['created_utc'].map(
            lambda t: dt.datetime.fromtimestamp(t))
    df_p = df_p.drop('created_utc', axis=1)                #Drop the column on timestamp format
    df_p = df_p.sort_values(by='datetime')                 #Sort the Row by datetime
    df_p["datetime"] = pd.to_datetime(df_p["datetime"])    #Convert timestamp format to datetime for data analysis


    df_p.to_csv(file_name, sep=',', # Save the dataset on a csv file for future analysis
                    header=True, index=False, columns=[
                'id', 'author', 'datetime',
                   'domain', 'url',
                   'title', 'num_comments', 'thumbnail'
            ])

    print(subreddit + " number of posts: " + str(len(df_p)))

In [5]:
"""FOR COMMENTS"""
def data_prep_comments(sub, start_time, end_time, filters, limit):
    if (len(filters) == 0):
        filters = ['id', 'author', 'created_utc', 'score',
                   'body']

    comments = list(api.search_comments(
        subreddit=sub,              
        after=start_time,       
        before=end_time,       
        filter=filters,    
        limit=limit))      
    return pd.DataFrame(comments) 

In [6]:
def get_subreddit_comments_and_save_to_csv(subreddit, day_start, month_start, year_start, day_end, month_end, year_end, file_name):
    
    start_time = int(dt.datetime(year_start, month_start, day_start).timestamp())
    end_time = int(dt.datetime(year_end, month_end, day_end).timestamp()) 
    filters = []       
    limit = None  

    """Here we are going to get subreddits for a brief analysis"""
    df_c = data_prep_comments(subreddit, start_time, end_time,filters,limit) 
    #df_c['datetime'] = pd.to_datetime(df_c['created_utc'], errors='coerce',unit='s',origin='1970-01-01')
    
    df_c['datetime'] = pd.to_datetime(df_c['created_utc'], errors='coerce')
    
    #df_c['datetime'] = df_c['created_utc'].map(lambda t: dt.datetime.fromtimestamp(t))
    
    df_c = df_c.drop('created_utc', axis=1)    
    df_c = df_c.sort_values(by='datetime')   
    #df_c["datetime"] = pd.to_datetime(df_c["datetime"]) 


    df_c.to_csv(file_name, sep=',', 
                    header=True, index=False, columns=[
                'id', 'author', 'datetime', 'score',
                   'body'
            ])

    print(subreddit + " number of posts: " + str(len(df_c)))

In [31]:
for subreddit_name in subreddits_for_analysis:
    list_of_vals = []
    for i in range(2018, 2023):
        for j in range(1,13):
            if (i==2018 and j>=2 and j!=3) or (i==2022 and j<=3) or (i!=2018 and i!=2022):
                str_i = str(i)
                str_j = str(j)
                df_2016 = pd.read_csv(f'new_folders\comments\comments_new\\{subreddit_name}\dataset_{subreddit_name}_comments_{str_i}_{str_j}.csv')
                print('len_' + str(i) + ': ' + str(len(df_2016)))
                
                list_of_vals.append(df_2016)

        df_combined = pd.DataFrame(columns = ['id', 'author', 'created_utc', 'score', 'body'])
        for v in list_of_vals:
            df_combined = df_combined.append(v)
    #df_combined = pd.concat([df_2016, df_2017, df_2018, df_2019, df_2020, df_2021])
        print('\nLen df_combined: '+ str(len(df_combined)))
        file_name = f'new_folders\comments_final\dataset_{subreddit_name}_comments_{str_i}.csv'
        df_combined.to_csv(file_name, sep=',', # Save the dataset on a csv file for future analysis
                        header=True, index=False, columns=[
                    'id','author','score','body','real_datetime'
                ])

len_2018: 40
len_2018: 17002
len_2018: 20621
len_2018: 14638
len_2018: 18361
len_2018: 16942
len_2018: 14378
len_2018: 11864
len_2018: 11678
len_2018: 17895

Len df_combined: 143419
len_2019: 18770
len_2019: 15856
len_2019: 17124
len_2019: 10602
len_2019: 10054
len_2019: 9934
len_2019: 10363
len_2019: 9462
len_2019: 8060
len_2019: 6924
len_2019: 6345
len_2019: 8252

Len df_combined: 275165
len_2020: 8624
len_2020: 7614
len_2020: 10064
len_2020: 7980
len_2020: 6739
len_2020: 8109
len_2020: 7276
len_2020: 7209
len_2020: 6483
len_2020: 5738
len_2020: 5853
len_2020: 5479

Len df_combined: 362333
len_2021: 4505
len_2021: 3442
len_2021: 2677
len_2021: 4060
len_2021: 5352
len_2021: 4656
len_2021: 4490
len_2021: 5582
len_2021: 5328
len_2021: 6007
len_2021: 5223
len_2021: 6152

Len df_combined: 419807
len_2022: 5062
len_2022: 4080
len_2022: 2977

Len df_combined: 431926
len_2018: 75
len_2018: 70782
len_2018: 62013
len_2018: 72251
len_2018: 83471
len_2018: 82130
len_2018: 72081
len_2018: 65461
l

In [9]:
subreddits_for_analysis = ['depression', 'offmychest', 'GetMotivated']#'happy', 'GetMotivated',

In [16]:
subreddits_for_analysis = ['happy', 'food', 'anxiety']

In [18]:
subreddits_for_analysis = ['happy', 'food', 'anxiety', 'offmychest', 'depression', 'GetMotivated']

In [10]:
def return_timestamp(dtt):
    curr_dt_1 = pd.to_datetime(dtt)
    final_timestamp = 1e9 * curr_dt_1.second + 1e3 * curr_dt_1.microsecond + curr_dt_1.nanosecond
    actual_date = pd.to_datetime(final_timestamp, unit='s')
    return actual_date

In [12]:
for subreddit_name in subreddits_for_analysis:
    for i in range(2018, 2023):
        for j in range(1,13):
            if (i==2018 and j>=2 and j!=3) or (i==2022 and j<=3) or (i!=2018 and i!=2022):
                str_i = str(i)
                str_j = str(j)
                file_name = f'comments\\{subreddit_name}\dataset_{subreddit_name}_comments_{str_i}_{str_j}.csv'
                df_p = pd.read_csv(file_name)
    
                df_p['real_datetime'] = df_p['datetime'].apply(lambda x: return_timestamp(x))
                df_p.drop('datetime', axis=1, inplace=True)
    
                df_p.to_csv(f'comments_new\\{subreddit_name}\dataset_{subreddit_name}_comments_{str_i}_{str_j}.csv')

In [ ]:
for subreddit in subreddits_for_analysis:
    get_subreddit_comments_and_save_to_csv(subreddit, 1, 5, 2020, 31, 5, 2020, f'dataset_{subreddit}_comments_2020_5.csv')

In [9]:
!pip install --upgrade pip setuptools wheel

  Using cached setuptools-62.0.0-py3-none-any.whl (1.1 MB)
  Using cached wheel-0.37.1-py2.py3-none-any.whl (35 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 58.1.0
    Uninstalling setuptools-58.1.0:
      Successfully uninstalled setuptools-58.1.0


In [7]:
!pip3 install --upgrade pip

  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\edin\\AppData\\Local\\Temp\\pip-uninstall-lpwaqmpe\\pip3.exe'
Consider using the `--user` option or check the permissions.



In [12]:
!python -m pip install -U symspellpy

  Using cached symspellpy-6.7.6-py3-none-any.whl (2.6 MB)
  Using cached editdistpy-0.1.3.tar.gz (57 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for editdistpy: filename=editdistpy-0.1.3-cp310-cp310-win_amd64.whl size=30060 sha256=606ff4065b36f7f6da210d6375c38e4d6a564d981a781f7fdc4eac449c502c70
  Stored in directory: c:\users\edin\appdata\local\pip\cache\wheels\88\6a\a6\a1283cc145323a1fb3d475bd158ee60b248ab1985230d266fc
Successfully built editdistpy


In [14]:
import pkg_resources
from symspellpy.symspellpy import SymSpell, Verbosity

In [15]:
# Set max_dictionary_edit_distance to avoid spelling correction

sym_spell = SymSpell(max_dictionary_edit_distance=0, prefix_length=7)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")

# term_index is the column of the term and count_index is the
# column of the term frequency

sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

# a sentence without any spaces

def fix_typo(input_term):
    result = sym_spell.word_segmentation(input_term)
    return result.corrected_string

In [17]:
df_anxiety_posts_1 = pd.read_csv('nutritional_database_combined_20_05.csv')

In [18]:
df_anxiety_posts_1.rename(columns = {'real_datetime':'datetime'}, inplace = True)

In [8]:
df_anxiety_posts = pd.read_csv('..\..\data\data\dataset_Anxiety_comments_2015_2022.csv')

In [38]:
df_anxiety_posts_1.head()

,description,food_class,food_category,serving_size,serving_size_unit,calories_serving_size,calories_100g,calories_unit,protein_serving_size,protein_100g,protein_unit,fat_serving_size,fat_100g,fat_unit,sugar_serving_size,sugar_100g,sugar_unit,sodium_serving_size,sodium_100g,sodium_unit,cholesterol_serving_size,cholesterol_100g,cholesterol_unit,carbohydrates_serving_size,carbohydrates_100g,carbohydrates_unit,fiber_serving_size,fiber_100g,fiber_unit,protein_calories,fat_calories,carb_calories
0,"Milk, human",Survey,Human milk,100,g,NaN,70.0,kcal,NaN,1.03,g,NaN,0.497,g,NaN,6.89,g,NaN,17.0,mg,NaN,14.0,mg,NaN,6.89,g,NaN,0.0,g,4.12,4.473,27.56
1,"Milk, NFS",Survey,"Milk, reduced fat",100,g,NaN,51.0,kcal,NaN,3.34,g,NaN,0.065,g,NaN,4.89,g,NaN,39.0,mg,NaN,8.0,mg,NaN,4.87,g,NaN,0.0,g,13.36,0.585,19.48
2,"Milk, whole",Survey,"Milk, whole",100,g,NaN,60.0,kcal,NaN,3.28,g,NaN,0.108,g,NaN,4.81,g,NaN,38.0,mg,NaN,12.0,mg,NaN,4.67,g,NaN,0.0,g,13.12,0.972,18.68
3,"Milk, low sodium, whole",Survey,"Milk, whole",100,g,NaN,61.0,kcal,NaN,3.10,g,NaN,0.128,g,NaN,4.46,g,NaN,3.0,mg,NaN,14.0,mg,NaN,4.46,g,NaN,0.0,g,12.40,1.152,17.84
4,"Milk, calcium fortified, whole",Survey,"Milk, whole",100,g,NaN,60.0,kcal,NaN,3.28,g,NaN,0.108,g,NaN,4.81,g,NaN,38.0,mg,NaN,12.0,mg,NaN,4.67,g,NaN,0.0,g,13.12,0.972,18.68


In [36]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [35]:
len(df_anxiety_posts_2)

7083

In [34]:
df_anxiety_posts_2[df_anxiety_posts_2['serving_size']!=100]

,Unnamed: 0,description,food_class,serving_size,serving_size_unit,calories,calories_unit,protein,protein_unit,fat,...,sugar,sugar_unit,sodium,sodium_unit,cholesterol,cholesterol_unit,carbohydrates,carbohydrates_unit,fiber,fiber_unit
1,0,"Milk, NFS",Survey,11.0,GM,51.0,kcal,3.34,g,0.065,...,4.89,g,39.0,mg,8.0,mg,4.87,g,0.0,g
20,0,"Kefir, NS as to fat content",Survey,50.0,GM,52.0,kcal,3.59,g,0.047,...,6.91,g,38.0,mg,5.0,mg,7.48,g,0.0,g
22,0,"Milk, dry, reconstituted, NS as to fat content",Survey,23.0,GM,32.0,kcal,3.20,g,0.003,...,4.60,g,51.0,mg,2.0,mg,4.60,g,0.0,g
23,0,"Milk, dry, reconstituted, whole",Survey,1.0,C,76.0,kcal,4.01,g,0.101,...,5.86,g,60.0,mg,15.0,mg,5.86,g,0.0,g
24,0,"Milk, dry, reconstituted, low fat (1%)",Survey,1.0,C,40.0,kcal,3.17,g,0.038,...,4.55,g,51.0,mg,4.0,mg,4.55,g,0.0,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7078,0,Tomatoes as ingredient in omelet,Survey,85.0,GM,25.0,kcal,1.11,g,0.094,...,3.42,g,6.0,mg,0.0,mg,5.48,g,1.6,g
7079,0,Other vegetables as ingredient in omelet,Survey,5.0,GM,29.0,kcal,3.46,g,0.175,...,2.73,g,6.0,mg,0.0,mg,4.81,g,1.4,g
7080,0,Vegetables as ingredient in curry,Survey,10.0,GM,52.0,kcal,1.81,g,0.064,...,3.25,g,19.0,mg,0.0,mg,11.60,g,2.2,g
7081,0,Sauce as ingredient in hamburgers,Survey,60.0,GM,271.0,kcal,1.34,g,13.500,...,13.10,g,845.0,mg,13.0,mg,17.10,g,0.6,g


In [32]:
df_anxiety_posts_2 = pd.read_csv('nutritional_values_survey_original.csv')

In [37]:
df_anxiety_posts_2.head()

,Unnamed: 0,description,food_class,serving_size,serving_size_unit,calories,calories_unit,protein,protein_unit,fat,fat_unit,sugar,sugar_unit,sodium,sodium_unit,cholesterol,cholesterol_unit,carbohydrates,carbohydrates_unit,fiber,fiber_unit
0,0,"Milk, human",Survey,100.0,GM,70.0,kcal,1.03,g,0.497,g,6.89,g,17.0,mg,14.0,mg,6.89,g,0.0,g
1,0,"Milk, NFS",Survey,11.0,GM,51.0,kcal,3.34,g,0.065,g,4.89,g,39.0,mg,8.0,mg,4.87,g,0.0,g
2,0,"Milk, whole",Survey,100.0,GM,60.0,kcal,3.28,g,0.108,g,4.81,g,38.0,mg,12.0,mg,4.67,g,0.0,g
3,0,"Milk, low sodium, whole",Survey,100.0,GM,61.0,kcal,3.10,g,0.128,g,4.46,g,3.0,mg,14.0,mg,4.46,g,0.0,g
4,0,"Milk, calcium fortified, whole",Survey,100.0,GM,60.0,kcal,3.28,g,0.108,g,4.81,g,38.0,mg,12.0,mg,4.67,g,0.0,g


In [24]:
df_anxiety_posts_1 = df_anxiety_posts_1.drop(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'Unnamed: 0.2'], axis=1)

In [25]:
df_anxiety_posts_1.keys()

Index(['description', 'food_class', 'food_category', 'serving_size',
       'serving_size_unit', 'calories_serving_size', 'calories_100g',
       'calories_unit', 'protein_serving_size', 'protein_100g', 'protein_unit',
       'fat_serving_size', 'fat_100g', 'fat_unit', 'sugar_serving_size',
       'sugar_100g', 'sugar_unit', 'sodium_serving_size', 'sodium_100g',
       'sodium_unit', 'cholesterol_serving_size', 'cholesterol_100g',
       'cholesterol_unit', 'carbohydrates_serving_size', 'carbohydrates_100g',
       'carbohydrates_unit', 'fiber_serving_size', 'fiber_100g', 'fiber_unit',
       'protein_calories', 'fat_calories', 'carb_calories'],
      dtype='object')

In [26]:
df_anxiety_posts_1.iloc[0]

description                   Milk, human
food_class                         Survey
food_category                  Human milk
serving_size                          100
serving_size_unit                       g
calories_serving_size                 NaN
calories_100g                        70.0
calories_unit                        kcal
protein_serving_size                  NaN
protein_100g                         1.03
protein_unit                            g
fat_serving_size                      NaN
fat_100g                            0.497
fat_unit                                g
sugar_serving_size                    NaN
sugar_100g                           6.89
sugar_unit                              g
sodium_serving_size                   NaN
sodium_100g                          17.0
sodium_unit                            mg
cholesterol_serving_size              NaN
cholesterol_100g                     14.0
cholesterol_unit                       mg
carbohydrates_serving_size        